# Assignment 22: Exceptions #

### Goals for this Assignment ###

By the time you have completed this assignment, you should be able to:

- Use `try` and `except` to catch exceptions
- Use `finally` to ensure that some execution will always happen around an exception
- Define your own exceptions by inheriting from the `Exception` class
- Throw your own exceptions using `raise`

## Step 1: Recover from `int` Reading a Non-Integer ##

### Background: Errors in Code ###

As a program executes, it's a somewhat common occurrence for things to go wrong.
For example:

- When reading an integer from the user, we discover that the input is not a valid integer
- When opening a file for reading, we discover that the file does not exist
- When performing division, we find that the divisor is zero
- When writing to a file, the disk runs out of space

When performing these sorts of actions, we can't just ignore these scenarios.
As such, we somehow need to indicate when these sort of problematic conditions occur.
One way to indicate this is to have related functions return special values, or _sentinel_ values, to indicate that something went wrong.
For example, for division, Python could have been designed so that if you attempted:

```python
x = 6 / 2
y = 5 / 0
```

...then `x` would be `3.0` (where division was successful and the result was stored in `x`), but `y` would be `None` (a special value indicating that division was unsuccessful).

This strategy for indicating issues does work, and it's commonly-used in some languages, most notably C.
However, it has a major problem: strictly speaking, we must always check if something went awry, even though it is expected that _usually_ it won't.
For example, say we intend to compute the sum of `x` and `y` in the above snippet.
Since `/` is not guaranteed to succeed (and returns `None` on failure), before we use `x` and `y`, we need to ensure that they don't hold `None`.
This means we'd need to do something like the following:

```python
x = 6 / 2
if x != None:
    y = 5 / 0
    if y != None:
        print(x + y)
    else:
        print("y's initialization lead to division by zero")
else:
    print("x's initialization lead to division by zero")
```

That is, we suddenly need a `if`/`else` to check if an error happened whenever we perform division.
This code is clearly much more bloated than:

```python
x = 6 / 2
y = 5 / 0
print(x + y)
```

The reality is unfortunately even worse than this.
For example, say we encapsulate this code in a separate function, and instead make the individual numbers parameters to that function, as shown below in the `compute_sum_with_division` function.

```python
def compute_sum_with_division(a, b, c, d):
    x = a / b
    if x != None:
        y = c / d
        if y != None:
            return x + y

    # will only reach this point if either of the divisions returned None
    return None
```

If a division by zero attempt is discovered within the `compute_sum_with_division` function, then this will ultimately return `None`.
This means that anything that calls `compute_sum_with_division` will _also_ need to check that the result from this is not `None`, even though the callers themselves don't attempt this operation.
This means that callers may need to propagate out the information that `compute_sum_with_division` returned `None` (or more specifically, _somewhere_ a division by zero was attempted).
This very quickly leads to situations where almost any function call can lead to potentially erroneous situations happening, requiring the sort of error-handling code seen in `compute_sum_with_division` to be propagated widely through the codebase.

Proper C code _will_ do this sort of propagation, but it's very common for programmers to optimistically assume that such situations will not happen.
It doesn't help that usually (but not always), such situations won't happen, so C programmers can _mostly_ get away with this.
However, if the program hits just the wrong input, such optimism can lead to very bizarre code behavior in C, usually requiring long debugging sessions to figure out not only where the error happened, but to refactor code to properly handle such errors.
In an ideal world, we would like something better.

### Background: Exceptions and Exception Handling ###

Enter _exceptions_.
The insight with exceptions is that usually there is some chunk of code _somewhere_ which properly handles the error.
However, with C, most of the error-related code merely propagates the information around that an error happened.
With exceptions, the idea is to just immediately jump to that error-handling code, without dealing with all the sort of propagation (or at least, make it so the programmer doesn't have to deal with this propagation).
This means exceptions allow you to write the optimistic code, like the original:

```python
x = 6 / 2
y = 5 / 0
print(x + y)
```

However, if a division by zero occurs, we will jump right to some code handling the error, which will be elsewhere.
This means we can write code that _mostly_ ignores errors, but still perform error-handling.

We've been seeing exceptions for awhile in this course, though they haven't been explained until now.
To show this, let's revisit the `int` function, which is used to convert some input to an integer representation.
When `int` is provided a string encoding an integer, then `int` can convert that string to an integer, as with:

In [1]:
print(int("123"))

123


However, if `int` is not given such a string, we instead see a `ValueError`, like so:

In [2]:
print(int("foo"))

ValueError: invalid literal for int() with base 10: 'foo'

Up until this point, we have explained this merely as the code crashing.
While this is technically true, and the crash is rooted in `int` being given a non-integer string, this glosses over an important detail.
Specifically, `int` _throws_ (or _raises_, in Python-specific speak) a `ValueError` _exception_.
Throwing this exception itself doesn't crash the code.
Instead, the code crashes because we do not have anything established which can handle this `ValueError` exception.
That is, if an exception is thrown which is not handled by anything, then the code will crash.

With this in mind, let's handle this `ValueError` exception.
In Python, this is done with `try`/`except`, and is illustrated in the code in the next cell:

In [3]:
print("before try")

try:
    print(int("foo"))
    print("after int call")
except ValueError:
    print("could not convert integer")

print("after try")


before try
could not convert integer
after try


If you run the cell above, you'll see that this code does **not** crash, and instead prints:

```
before try
could not convert integer
after try
```

To explain why, `try` is used to indicate that some given piece of code _might_ throw an exception.
The specific code is in the body of `try` (i.e., that code is indented in).
Sure enough, looking at the code in `try`'s body, we can see the offending `int("foo")` call.
After the `try`, at the same indent level as the `try`, we can then define one or more `except`s.
Each `except` specifies the name of an exception, and each `except` has its own separate body.
The `except` in the above code, in plain English, says that if the code within the body of `try` throws a `ValueError` exception, then the bode in the body of the `except` should be executed.

With all this in mind, if we run the above code, the following happens:

- Python sees the `try`, as well as the `try`'s `except`.  Python will record that this particular `except` handles `ValueError`s.
- Python will execute the body of `try`.
- The `int("foo")` expression is executed, which throws a `ValueError` exception.
- Python sees a `ValueError` exception has been thrown, but it's within a `try`.  Python sees that we had a particular `except` for `ValueError` exceptions established, and so execution jumps to the body of the specific `except` for `ValueError` exceptions.
- Once the body of the `except` is evaluated, execution moves to the end of the whole `try`/`except`, bringing us to the next (and final) statement: `print("after try")`

With all of this in mind, we can establish that some given code needs to run in case a particular exception occurs.

Perhaps most importantly, the only times we need to use `try`/`except` in Python is when we establish the code to handle an exception.
If all we want to do is propagate an exception, we don't need to do anything at all.
To illustrate this, let's revisit the `compute_sum_with_division` function from before.
Let's say division by zero now throws an exception, as opposed to returning the sentinel value `None` (and in fact, Python does throw a `ZeroDivisionError` exception if division by zero is attempted).
With this in mind, if we don't plan to handle the division by zero case within `compute_sum_with_division`, we can instead write:

```python
def compute_sum_with_division(a, b, c, d):
    x = a / b
    y = c / d
    return x + y
```

...that is, there is nothing explicitly stated for division by zero at all.
If `a / b` triggers division by zero, then this expression will immediately throw an exception, causing execution to jump out of the `compute_sum_with_division` function entirely.
If, however, `a / b` does **not** trigger division by zero, then execution will proceed forward as normal.
The same logic applies for the later `c / d` expression.
This means that code using exceptions can largely ignore the fact that they exist at all, except at the precise moments when we want to handle exceptions.

So with all this in mind, what exactly is an exception itself?
In Python, exceptions are just objects, specifically objects inheriting from the `Exception` class.
`ValueError` is a class that comes predefined for us, but we are permitted to define any of our own.
We will get into this more in a later step.

### Try this Yourself ###

Define a function named `parse_integer_or_else`, which will take two parameters:

1. Some value, intended to be a string encoding an integer
2. An integer

`parse_integer_or_else` should attempt to parse its first input as an integer, using `int`.
If successful, then `parse_integer_or_else` should return the parsed integer.
However, if `int` was unable to parse the input as an integer, then `parse_integer_or_else` should instead return the value of its second parameter.

Define your `parse_integer_or_else` function in the next cell.
Leave the calls in place in order to test your code.

In [5]:
# Define your parse_integer_or_else function below.
# Leaves the calls in place in order to test your code.
def parse_integer_or_else(x,y=0):
    try:
        return int(x)

    except ValueError:
        return int(y)
        


print(parse_integer_or_else("123", 5) + 1) # should print 124
print(parse_integer_or_else("14", 2) + 2) # should print 16
print(parse_integer_or_else("foo", 3) + 1) # should print 4

124
16
4


## Step 2: Use `finally` to Guarantee Some Computation Happens on an Exception ##

### Background: `finally` ###

Sometimes you need to guarantee that some bit of code gets executed _even if_ an exception is thrown, even if you don't have a handler defined for an exception.
For example, in assignment 17, you worked with files, which must (in order):

1. Be opened
2. Be written to / read from
3. Be closed

For reasons beyond our scope, closing the file when you're done is very important, especially if you are writing to a file.
Without it, the file you "wrote" to might appear to be fine, or it may only contain part of what you wrote, become somehow corrupted, or perhaps not even appear at all.
In the context of exceptions, this can be particularly problematic.
For example, consider the following code.
Note we are _not_ using `with...as` in this example, meaning we must explicitly call `close`; there will be more on that in a bit.

```python
file_handle = open("some_file.txt", "w")
file_handle.write("foo")
file_handle.write(str(int("foo") + 1))
file_handle.close()
```

While the above code may successfully open `"some_file.txt"`, and even write `"foo"` to said file, the subsequent line would trigger a `ValueError` exception.
We can partially handle this by catching a `ValueError` exception, as with:

```python
try:
    file_handle = open("some_file.txt", "w")
    file_handle.write("foo")
    file_handle.write(str(int("foo") + 1))
    file_handle.close()
except ValueError:
    print("could not convert integer")
    file_handle.close()
```

The above code will now call `file_handle.close()` whether or not a `ValueError` is thrown.
(Note for those coming from other languages: `file_handle` _will_ be in scope in the `except`, despite typical scoping rules.)
However, it turns out we have only half-fixed this issue, and introduced another problem in the process, namely:

- If any exception _other than_ `ValueError` is thrown, we will still have the same problem of failing to close `file_handle`.
- We duplicated the `close` operation, and for each kind of exception which could be thrown, we'd need to duplicate it again.

To address situations like this, in addition to `except`, we also have `finally`.
The code inside `finally` is executed after the body of the `try` and any `except`s are executed.
This is shown in the cell below:

In [6]:
try:
    file_handle = open("some_file.txt", "w")
    file_handle.write("foo")
    file_handle.write(str(int("foo") + 1))
except ValueError:
    print("could not convert integer")
finally:
    file_handle.close()

could not convert integer


If you run the above cell, it will create a file named `"some_file.txt"` in the same directory as the notebook.
`"foo"` will be written to the file, and the file will be successfully closed.
The cell will also print out `"could not convert integer"`, indicating that the `ValueError` exception is thrown.
Importantly, the `file_handle.close()` is now guaranteed to run whether or not an exception gets thrown, or even if we catch an exception, and we do not need to duplicate this code, either.

Admittedly, in assignment 17, we did not call `close`, and instead used `with...as`.
`with...as` creates something called a [context manager](https://docs.python.org/3/reference/datamodel.html#context-managers), which are used for the express purpose of ensuring that some action is performed automatically once some block of code has finished executing.
Context managers solve a similar issue as `finally` here, but without even needing to write the `close`.

### Try this Yourself ###

The function in the next cell duplicates a `print` depending on whether or not a `ZeroDivisionError` exception is thrown.
Rewrite this code to instead use `finally`.
Leave the calls in place in order to test your code.

In [10]:
# Rewrite does_division to use finally, and avoid duplicating the print
def does_division(x, y):
    try:
        result = x / y
        #print("division attempt complete")
        #return result
    except ZeroDivisionError:
        #print("division attempt complete")
        #return 0
        result = 0
    finally:
        print("division attempt complete")
    return result

print(does_division(6, 2))
# above statement should print:
# division attempt complete
# 3.0

print(does_division(8, 0))
# above statement should print:
# division attempt complete
# 0

division attempt complete
3.0
division attempt complete
0


## Step 3: Define and Throw your own Exceptions ##

### Background: Defining and Throwing Custom Exceptions ###

You can define your own exceptions by inheriting from the `Exception` class, as illustrated in the cell below:

In [11]:
class MyException(Exception):
    pass

Because `MyException` (and any exception type you want) is a class, you can do anything with exceptions that you can do with objects.
In practice, it's common for people to put extra information in an exception to record information about what specifically went wrong; we will see this momentarily.

Exceptions can be thrown with the `raise` statement.
For example, consider the code in the following cell:

In [13]:
raise MyException()

MyException: 

The above cell will create an instance of the `MyException` class, and immediately throw it with `raise`.
Because this code is not enclosed in a `try` with an `except` handling `MyException`, the cell crashes.
We don't see this crash with the `try`, as with:

In [14]:
try:
    raise MyException()
except MyException:
    print("caught exception")

caught exception


The one issue with the `except` as written is that this merely checks to see if a `MyException` instance was thrown.
The actual instance is discarded.
With `MyException`, this is not that big of a deal, since there isn't any information that we put into this exception.
However, consider the `MyExceptionWithMessage` exception defined in the next cell:

In [15]:
class MyExceptionWithMessage(Exception):
    def __init__(self, message):
        self.message = message

In this case, there is a `message` that is stored inside of the exception instance itself.
This message could be used to encode some extra information about what exactly went wrong.
In order to access this message, if the exception is thrown, we would need a handle on the instance of the exception.
This can be done with `as`, like so:

In [16]:
try:
    raise MyExceptionWithMessage("hello")
except MyExceptionWithMessage as e:
    print(e.message)

hello


As shown, `as` can be put after the name of the type of exception we want to catch, and this is followed by the name of a new variable, in this case `e`.
The name of the variable can be wahtever you want it to be.
This means that `e` will be bound to the specific `MyExceptionWithMessage` instance thrown by `raise`, hence the code above printing out `"hello"`.

### Try this Yourself ###

In the next cell, you'll need to define a `check_input` function, which will take a single parameter.
From there, `check_input` will do the following:

- If the parameter was the string `"foo"`, then `check_input` will throw a `GotFoo` exception.
- If the parameter was the string `"bar"`, then `check_input` will throw a `GotBar` exception.
- If the parameter could be parsed with `int`, then `check_input` will throw a `GotInteger` exception.  The actual value returned by `int` will be stored in a field named `value` on the `GotInteger` exception instance.
- If the parameter was anything else, the parameter should be returned as-is

In order to make your `check_input` function work, you will also need to define `GotFoo`, `GotBar`, and `GotInteger` exceptions.

The expected output of the next cell is as follows:

```
Got foo
Got bar
17
3
apple
pear
```

In [24]:
# Define your exception classes and check_input function here.

class GotFoo(Exception):
    pass

class GotBar(Exception):
    pass

class GotInteger(Exception):
    def __init__(self, value, *args):
        Exception.__init__(self, *args) 
        self.value = value


def check_input(input_value):
    if isinstance(input_value, str):
        if input_value == "foo":
            raise GotFoo
        elif input_value == "bar":
            raise GotBar
    elif isinstance(input_value, int):
        raise GotInteger(input_value)
    return input_value
   


# Leave the code below in place for testing.

try:
    check_input("foo")
except GotFoo:
    print("Got foo")

try:
    check_input("bar")
except GotBar:
    print("Got bar")

try:
    check_input(17)
except GotInteger as e:
    print(e.value)

try:
    check_input(3)
except GotInteger as e:
    print(e.value)

print(check_input("apple"))
print(check_input("pear"))

Got foo
Got bar
17
3
apple
pear


## Step 4: Submit via Canvas ##

Be sure to **save your work**, then log into [Canvas](https://canvas.csun.edu/).  Go to the COMP 502 course, and click "Assignments" on the left pane.  From there, click "Assignment 22".  From there, you can upload the `22_exceptions.ipynb` file.

You can turn in the assignment multiple times, but only the last version you submitted will be graded.